In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# Set up the tools

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

In [4]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

# Set up the model

In [11]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
model = ChatOpenAI(temperature=0, streaming=True)

In [12]:
from langchain_core.utils.function_calling import convert_to_openai_tool

tools = [convert_to_openai_tool(t) for t in tools]
model = model.bind_tools(tools)

# Define the nodes

In [32]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage


# Define the function that determines whether to continue or not
def should_continue(messages):
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
async def call_model(messages):
    response = await model.ainvoke(messages)
    return response


# Define the function to execute tools
async def call_tool(messages):
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = await tool_executor.ainvoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return function_message

# Define the edges

In [33]:
from langgraph.graph import MessageGraph, END

# Define a new graph
workflow = MessageGraph()

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

# Streaming LLM Tokens

In [34]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="马斯克投资了哪些公司？")]
async for event in app.astream_events(inputs, version="v1"):
    kind = event["event"]
    print(kind)
    if kind == "on_chain_stream":
        print(event["data"])
        chunk = event["data"]["chunk"]
        for x in chunk:
            if "content" in chunk:
                print(chunk['content'], end="|", flush=True)
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

on_chain_start
on_chain_start
on_chain_stream
{'chunk': [HumanMessage(content='马斯克投资了哪些公司？')]}
on_chain_end
on_chain_start
on_chain_start
on_chain_stream
{'chunk': [HumanMessage(content='马斯克投资了哪些公司？')]}
on_chain_start
on_chain_stream
{'chunk': [HumanMessage(content='马斯克投资了哪些公司？')]}
on_chain_stream
{'chunk': [HumanMessage(content='马斯克投资了哪些公司？')]}
on_chain_end
on_chain_end
on_chain_end
on_chain_start
on_chain_start
on_chain_stream
{'chunk': [HumanMessage(content='马斯克投资了哪些公司？')]}
on_chain_start
on_chain_end
on_chain_stream
{'chunk': AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_VoVPfxqE8SSp4ib4vxRkds6D', 'function': {'arguments': '{"query":"Elon Musk investments"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})}
on_chain_start
on_chain_stream
{'chunk': AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_VoVPfxqE8SSp4ib4vxRkds6D', 'function': {'arguments': '{"query":"Elon Musk investments"}', 'name': 'tavily_se

In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict, Union

from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import BaseMessage, FunctionMessage, ToolMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_tool

from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_tool_calling_executor(
    model: LanguageModelLike, tools: Union[ToolExecutor, Sequence[BaseTool]]
):
    if isinstance(tools, ToolExecutor):
        tool_executor = tools
        tool_classes = tools.tools
    else:
        tool_executor = ToolExecutor(tools)
        tool_classes = tools
    model = model.bind(tools=[convert_to_openai_tool(t) for t in tool_classes])

    # We create the AgentState that we will pass around
    # This simply involves a list of messages
    # We want steps to return messages to append to the list
    # So we annotate the messages attribute with operator.add
    class AgentState(TypedDict):
        messages: Annotated[Sequence[BaseMessage], operator.add]

    # Define the function that determines whether to continue or not
    def should_continue(state: AgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there is no function call, then we finish
        if "tool_calls" not in last_message.additional_kwargs:
            return "end"
        # Otherwise if there is, we continue
        else:
            return "continue"

    # Define the function that calls the model
    def call_model(state: AgentState):
        messages = state["messages"]
        response = model.invoke(messages)
        # We return a list, because this will get added to the existing list
        return {"messages": [response]}

    async def acall_model(state: AgentState):
        messages = state["messages"]
        response = await model.ainvoke(messages)
        # We return a list, because this will get added to the existing list
        return {"messages": [response]}

    # Define the function to execute tools
    def _get_actions(state: AgentState):
        messages = state["messages"]
        # Based on the continue condition
        # we know the last message involves a tool call
        last_message = messages[-1]
        # We construct an AgentAction from each of the tool_calls
        return (
            [
                ToolInvocation(
                    tool=tool_call["function"]["name"],
                    tool_input=json.loads(tool_call["function"]["arguments"]),
                )
                for tool_call in last_message.additional_kwargs["tool_calls"]
            ],
            [
                tool_call["id"]
                for tool_call in last_message.additional_kwargs["tool_calls"]
            ],
        )

    def call_tool(state: AgentState):
        actions, ids = _get_actions(state)
        # We call the tool_executor and get back a response
        responses = tool_executor.batch(actions)
        # We use the response to create a FunctionMessage
        tool_messages = [
            ToolMessage(content=str(response), tool_call_id=id)
            for response, id in zip(responses, ids)
        ]
        # We return a list, because this will get added to the existing list
        return {"messages": tool_messages}

    async def acall_tool(state: AgentState):
        actions, ids = _get_actions(state)
        # We call the tool_executor and get back a response
        responses = await tool_executor.abatch(actions)
        # We use the response to create a FunctionMessage
        tool_messages = [
            ToolMessage(content=str(response), tool_call_id=id)
            for response, id in zip(responses, ids)
        ]
        # We return a list, because this will get added to the existing list
        return {"messages": tool_messages}

    # Define a new graph
    workflow = StateGraph(AgentState)

    # Define the two nodes we will cycle between
    workflow.add_node("agent", RunnableLambda(call_model, acall_model))
    workflow.add_node("action", RunnableLambda(call_tool, acall_tool))

    # Set the entrypoint as `agent`
    # This means that this node is the first one called
    workflow.set_entry_point("agent")

    # We now add a conditional edge
    workflow.add_conditional_edges(
        # First, we define the start node. We use `agent`.
        # This means these are the edges taken after the `agent` node is called.
        "agent",
        # Next, we pass in the function that will determine which node is called next.
        should_continue,
        # Finally we pass in a mapping.
        # The keys are strings, and the values are other nodes.
        # END is a special node marking that the graph should finish.
        # What will happen is we will call `should_continue`, and then the output of that
        # will be matched against the keys in this mapping.
        # Based on which one it matches, that node will then be called.
        {
            # If `tools`, then we call the tool node.
            "continue": "action",
            # Otherwise we finish.
            "end": END,
        },
    )

    # We now add a normal edge from `tools` to `agent`.
    # This means that after `tools` is called, `agent` node is called next.
    workflow.add_edge("action", "agent")

    # Finally, we compile it!
    # This compiles it into a LangChain Runnable,
    # meaning you can use it as you would any other runnable
    return workflow.compile()
